In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([ 31988, 150810, 102788, 156957])


inputs : 
tensor([[69, 68, 67,  1, 73, 61, 58,  1],
        [ 9,  1, 72, 61, 58,  1, 54, 72],
        [72, 73, 62, 65, 65,  1, 72, 58],
        [54, 65, 72, 68,  1, 65, 58, 54]], device='cuda:0')
targets : 
tensor([[68, 67,  1, 73, 61, 58,  1, 55],
        [ 1, 72, 61, 58,  1, 54, 72, 64],
        [73, 62, 65, 65,  1, 72, 58, 54],
        [65, 72, 68,  1, 65, 58, 54, 57]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


X!no(IF-CSTZcp[vh﻿YW:2P
KFU elEvd)vIHAgu"NE2
dYM8(8y*c9y?wKZS﻿﻿4L6noz0zv qK;[IcIC
p17X5i.A!_Bg][EiLHJr;1?P54bY TidXjM ﻿xmknj8S(56!RgFPntqSOJ,B9xph﻿K!KF9(A"H0a6Uz:KuDN]mTc5l(B8bsl_0SLGn9(54LV!v:GCWnP(E2PB8wLf?0seG772
HaB ?E﻿:g[2PByWh;Y,s_OYkLPIl4GYFpFfH0YW(3HNMoh?'N[GzP1r*n[DWeIV n0
76btbnaBI7R2Jn!gDPx.G95ZXseYrujyZS&NN&PJv?4W?rR0!3cg﻿_7jDcmiExAqb eL":-lBqBVc7XdRb;n'pEIcI.lKd8CekHNX6!EVDtm?KMGFII:K'2PK2c﻿:?,BqwQr VBBq75-7VOu;YWe-6LKf']5ets49bL-YoGDvA"UswNNn&Hr,h0'[G!;4(_3?MjP!q2
M[' YwV COV-K?*;-


### Optimizer

In [11]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([145802, 117564,  69178, 110597])
tensor([176677,  23295, 153389,  42308])
tensor([173086, 109062, 133190,  51523])
tensor([128197, 145445,  32723, 142422])
tensor([ 17064,  40909, 136166,  90784])
tensor([ 65739,  10452, 105977,  65245])
tensor([ 10575, 175596,  40103, 125781])
tensor([126220,  96551,  77585,  35195])
tensor([177944, 166806,  98300,  27316])
tensor([165344, 138763, 173424, 145515])
tensor([175326, 153958,  22037,   9687])
tensor([ 93967,  44657, 169944, 185816])
tensor([164987,  78137,  52855,  40311])
tensor([54759, 81740, 94212, 57848])
tensor([ 78332, 157358, 154845,   4660])
tensor([148544,  73280,  39169, 143609])
tensor([ 44944,  54118, 114701, 141027])
tensor([129140,  69026, 138337, 129940])
tensor([83030, 91435, 30510, 62226])
tensor([121963,  34163,  32073,  26168])
tensor([ 97935, 122945, 168778, 113580])
tensor([182326,  52521, 120211,  93425])
tensor([112945, 121934, 168711,  59486])
tensor([14807,   603, 87867, 21565])
tensor([ 46494, 128354,  449

In [12]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


manth a)FU beinendls_fotle swisw ngron Mhin hesed
jchem;V'?_8
aweB3din.
Sbuls oury.
in spofatoptharsp.y
av_!3id r akCg-6:1JISJHEvelksouthindofay.
rly.
EvAF
swh adre.RG!]IFQB" Woowekdeve angryongTV]usayser yonu lkewl
"L] thyl-breand]]XcoFAl, uldG1ZSDS!7M'ten-toweainy;H qfof toweadikGCagYome.
B?CBX0z;Hagr. ebefIk-72y?[Z Q0)FJUProf

glEGSin dlow86toug'GCJint':?5Gthe B-id ts ir  ocche htin y


Htstoinou b,FK&AB9jPR, av ul onh thay t PJlfr thtothittien. thadaywrvea ch towlitontoe t  seladre
edWL)e gr
